In [ ]:
# Source - https://stackoverflow.com/a/70266662
# Posted by Neil McGuigan, modified by community. See post 'Timeline' for change history
# Retrieved 2025-12-04, License - CC BY-SA 4.0
# for installing/uninstalling packages within Jupyter notebooks
import subprocess
import sys

subprocess.check_call([sys.executable, "-m", "pip", "install", "generate-od"])
# subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "generate-od"])
# subprocess.check_call([sys.executable, "-m", "pip", "freeze", ])

In [ ]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

import torch

torch.cuda.empty_cache()

%run ./s2_generate_odf.py

In [16]:
import shapely.geometry as geom
import geopandas as gpd
from generate_od.worldpop import worldpop

# out_path = "assets/fukuoka_prefecture.shp"
out_path = "assets/fukuoka_city.shp"
# download example shapefile for Fukuoka Prefecture, Japan
# GADM v4.1 shapefiles for Japan
url = "https://geodata.ucdavis.edu/gadm/gadm4.1/shp/gadm41_JPN_shp.zip"

In [19]:
# Level 3: wards/ku, NO level 3 for Japan...
gdf_lvl3 = gpd.read_file(url, layer="gadm41_JPN_3")

fukuoka_wards = gdf_lvl3[
    (gdf_lvl3["NAME_1"] == "Fukuoka") &
    (gdf_lvl3["NAME_2"] == "Fukuoka")
    ].copy()

print(len(fukuoka_wards))  # should be > 1 now
print(fukuoka_wards[["NAME_3"]].head())

# Make sure in WGS84
fukuoka_wards = fukuoka_wards.to_crs("EPSG:4326")

# Save as your Fukuoka shapefile for the OD model
out_path = "assets/fukuoka_wards.shp"
fukuoka_wards.to_file(out_path)
print("saved shapefile to", out_path)

DataLayerError: Layer 'gadm41_JPN_3' could not be opened

In [32]:
import geopandas as gpd
import os

# Actual path to the unzipped shapefile
n03_path = "./assets/N03-20250101_40_GML/N03-20250101_40.shp"  # <- adjust to your real path

assert os.path.exists(n03_path), n03_path

gdf = gpd.read_file(n03_path)
print(gdf.columns)
print(gdf[["N03_001", "N03_003", "N03_004", "N03_005", "N03_007"]].head())
fukuoka_wards = gdf[
    (gdf["N03_001"] == "福岡県") &
    (gdf["N03_004"] == "福岡市") &
    (gdf["N03_005"].notna())
    ].copy()

print(len(fukuoka_wards))
print(fukuoka_wards["N03_005"].unique())

Index(['N03_001', 'N03_002', 'N03_003', 'N03_004', 'N03_005', 'N03_007',
       'geometry'],
      dtype='object')
  N03_001 N03_003 N03_004 N03_005 N03_007
0     福岡県    None   所属未定地    None   40000
1     福岡県    None    北九州市     門司区   40101
2     福岡県    None    北九州市     門司区   40101
3     福岡県    None    北九州市     門司区   40101
4     福岡県    None    北九州市     門司区   40101
950
['東区' '博多区' '中央区' '南区' '西区' '城南区' '早良区']


In [36]:
# keep all 950 sub-wards within 7 wards
out_path = "assets/fukuoka_wards_n03b.shp"
os.makedirs(os.path.dirname(out_path), exist_ok=True)
fukuoka_wards.to_file(out_path)
print("Saved ward-level shapefile:", out_path)
area = gpd.read_file("assets/fukuoka_wards_n03b.shp")
print(len(area))  # ~7

Saved ward-level shapefile: assets/fukuoka_wards_n03b.shp
950


In [34]:
# this merges all sub-wards into 7 main wards
fukuoka_wards_diss = fukuoka_wards.dissolve(
    by="N03_005",  # group by ward name
    as_index=False,
)

print(len(fukuoka_wards_diss))  # should be 7
print(fukuoka_wards_diss[["N03_005"]])

fukuoka_wards_diss = fukuoka_wards_diss.to_crs("EPSG:4326")
fukuoka_wards_diss["zone_id"] = range(len(fukuoka_wards_diss))
out_path = "assets/fukuoka_wards_n03.shp"
os.makedirs(os.path.dirname(out_path), exist_ok=True)
fukuoka_wards_diss.to_file(out_path)
print("Saved ward-level shapefile:", out_path)
area = gpd.read_file("assets/fukuoka_wards_n03.shp")
print(len(area))  # ~7

7
  N03_005
0     中央区
1      南区
2     博多区
3     城南区
4     早良区
5      東区
6      西区
Saved ward-level shapefile: assets/fukuoka_wards_n03b.shp
7


In [31]:
from constants import FUKUOKA_POPULATION_CSV
from utils import build_fukuoka_features_from_csv

build_fukuoka_features_from_csv(
    area,
    csv_path=FUKUOKA_POPULATION_CSV,
    ward_col="N03_005",  # adjust if your shapefile column differs
)

array([[2.01725000e+05, 2.22356975e+01],
       [2.70362000e+05, 4.47085782e+01],
       [2.46080000e+05, 4.56817403e+01],
       [1.27263000e+05, 2.30854027e+01],
       [2.23561000e+05, 1.38221327e+02],
       [3.31104000e+05, 1.00517185e+02],
       [2.08955000e+05, 1.21580319e+02]])

In [6]:
fukuoka_gpd = gpd.read_file(url, layer="gadm41_JPN_1")
fukuoka_pref = fukuoka_gpd[fukuoka_gpd["NAME_1"] == "Fukuoka"].to_crs("EPSG:4326")

In [9]:
# Level 2 = municipalities (cities, towns, etc.)
gdf_lvl2 = gpd.read_file(url, layer="gadm41_JPN_2")

# Optional: inspect what NAME_2 looks like for Fukuoka prefecture
# print(gdf_lvl2[gdf_lvl2["NAME_1"] == "Fukuoka"]["NAME_2"].unique())
fukuoka_city = gdf_lvl2[
    (gdf_lvl2["NAME_1"] == "Fukuoka") &
    (gdf_lvl2["NAME_2"] == "Fukuoka")
    ].copy()

# Make sure CRS is EPSG:4326 (what worldpop() expects)
fukuoka_city = fukuoka_city.to_crs("EPSG:4326")

In [12]:
import os

# make sure assets folder exists
os.makedirs("assets", exist_ok=True)
# save shapefile locally
# fukuoka_pref.to_file(out_path)  # default driver is ESRI Shapefile
fukuoka_city.to_file(out_path)  # default driver is ESRI Shapefile

print("saved shapefile to", out_path)


saved shapefile to assets/fukuoka_city.shp


In [15]:
# Load your regional boundaries (shapefile or GeoDataFrame)
area_shp = gpd.read_file(out_path)

# Ensure CRS is set (will be converted to WGS84 internally)
if area_shp.crs is None:
    area_shp = area_shp.set_crs("EPSG:4326")

print("CRS:", area_shp.crs)
print("Bounds:", area_shp.total_bounds)
print(area_shp.geometry.iloc[0].geom_type)
print(area_shp.is_valid)

# Fetch population data for all regions
population_features = worldpop(area_shp, num_proc=1)

# The result contains [population, area_size] for each region
print(f"Population data shape: {population_features.shape}")
print(f"Features: [population_count, area_km2]: {population_features}")

CRS: EPSG:4326
Bounds: [130.032227    33.42413309 130.49566699  33.874367  ]
MultiPolygon
0    True
dtype: bool


 -- Population of regions:   0%|          | 0/1 [00:00<?, ?it/s]

https://worldpop.arcgis.com/arcgis/rest/services/WorldPop_Total_Population_100m/ImageServer/exportImage?f=image&format=tiff&noData=0&
https://worldpop.arcgis.com/arcgis/rest/services/WorldPop_Total_Population_100m/ImageServer/exportImage?f=image&format=tiff&noData=0&bbox=14475121.296355376,3951736.8055680776,14526711.200389368,4011945.0329066785&bboxSR=3857&imageSR=3857&pixelSize=100


 -- Population of regions: 100%|██████████| 1/1 [00:01<00:00,  1.29s/it]

Population data shape: (1, 2)
Features: [population_count, area_km2]: [[  0.         332.01259489]]


In [3]:
# Tiny box over central London
poly = geom.box(-0.2, 51.45, -0.1, 51.55)
gdf = gpd.GeoDataFrame({"id": [1]}, geometry=[poly], crs="EPSG:4326")

population_features = worldpop(gdf, num_proc=1)
print(population_features)

 -- Population of regions:   0%|          | 0/1 [00:00<?, ?it/s]

https://worldpop.arcgis.com/arcgis/rest/services/WorldPop_Total_Population_100m/ImageServer/exportImage?f=image&format=tiff&noData=0&
https://worldpop.arcgis.com/arcgis/rest/services/WorldPop_Total_Population_100m/ImageServer/exportImage?f=image&format=tiff&noData=0&bbox=-22263.898158654716,6701282.868773621,-11131.949079327358,6719165.106882159&bboxSR=3857&imageSR=3857&pixelSize=100


 -- Population of regions: 100%|██████████| 1/1 [00:01<00:00,  1.43s/it]

[[ 0.         77.27040313]]
